# <font color='#6629b2'>Predicting sentiment ratings with RNN using Keras</font>


## <font color='#6629b2'>Study Material</font>
- https://web.stanford.edu/~jurafsky/slp3/9.pdf
- https://www.youtube.com/watch?v=UNmqTiOnRfg
- https://www.youtube.com/watch?v=WCUNPb-5EYI
- https://www.youtube.com/watch?v=OuYtk9Ymut4

## <font color='#6629b2'>Dataset</font>

The Large Movie Review Datasetconsists of 50,000 movie reviews from [IMDB](http://www.imdb.com/). The ratings are on a 1-10 scale, but the dataset specifically contains "polarized" reviews: positive reviews with a rating of 7 or higher, and negative reviews with a rating of 4 or lower. There are an equal number of positive and negative reviews. 

You can download the dataset from: https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [20]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [21]:
'''Load the dataset into variable "reviews". You can truncate the dataset to keep a few hundred records if it's
    taking too long to process/train. Keep in mind, bigger the dataset, higher the accuracy score!
'''
import pandas as pd 
import numpy as np

reviews = pd.read_csv('/content/gdrive/MyDrive/Synapse_LP/NLP_Week3/IMDB Dataset.csv',nrows=1000)
reviews

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
995,Nothing is sacred. Just ask Ernie Fosselius. T...,positive
996,I hated it. I hate self-aware pretentious inan...,negative
997,I usually try to be professional and construct...,negative
998,If you like me is going to see this in a film ...,negative


In [22]:
#Split the data into train and test

from sklearn.model_selection import train_test_split
x_review, y_review, x_sentiments, y_sentiments = train_test_split(reviews['review'], reviews['sentiment'], test_size=0.25, random_state=42)

In [23]:
# convert the above train test datasets into a pandas dataframe. You shoud have 4 dataframes.
# Name them train_reviews, test_reviews, train_sentiment, test_sentiment.
# In train_sentiment and test_sentiment, convert "positive" to 1 and "negative" to 0

a_sentiments = (x_sentiments.replace({'positive': 1, 'negative': 0})).values
b_sentiments = (y_sentiments.replace({'positive': 1, 'negative': 0})).values

train_reviews = pd.DataFrame(x_review, columns=['review'])
test_reviews = pd.DataFrame(y_review, columns=['review'])
train_sentiment = pd.DataFrame(a_sentiments, columns=['sentiment'])
test_sentiment = pd.DataFrame(b_sentiments, columns=['sentiment'])

print(train_reviews.shape, train_sentiment.shape, test_reviews.shape, test_sentiment.shape)
test_sentiment

(750, 1) (750, 1) (250, 1) (250, 1)


,sentiment
0,1
1,1
2,0
3,0
4,1
...,...
245,1
246,0
247,0
248,0


In [24]:
train_sentiment.head()

,sentiment
0,0
1,0
2,1
3,1
4,0


## <font color='#6629b2'>Preparing the data</font>

###  <font color='#6629b2'>Tokenization</font>

The first preprocessing step is to tokenize each of the reviews into (lowercased) individual words, since the models will encode the reviews at the word level (rather than subword units like characters, for example). For this we'll use [spaCy](https://spacy.io/), which is a fast and extremely user-friendly library that performs various language processing tasks. Once you load a spaCy model for a particular language, you can provide any text as input to the model, e.g. encoder(text) and access its linguistic features.

In [25]:
'''Lowercase and tokenise all the reviews in train_reviews using spacy'''

import spacy
encoder = spacy.load('en_core_web_sm')

def text_to_tokens(text_seqs):
    #complete this function that lowers and tokenizes the reviews
    token_seqs = [[word.lower_ for word in encoder(data)] for data in text_seqs]
    return token_seqs
    # tokenise_list = []
    # for data in text_seqs:
    #   low = data.lower()
    #   token = encoder(low)
    #   tokenise_list.append(token)
    # return tokenise_list

train_reviews['Tokenized_Review'] = text_to_tokens(train_reviews['review'])
train_reviews[['review','Tokenized_Review']][:10]

,review,Tokenized_Review
82,"Oh God, I must have seen this when I was only ...","[oh, god, ,, i, must, have, seen, this, when, ..."
991,I dunno sometimes...you try and try and try to...,"[i, dunno, sometimes, ..., you, try, and, try,..."
789,This little two-person movie is actually much ...,"[this, little, two, -, person, movie, is, actu..."
894,After several extremely well ratings to the po...,"[after, several, extremely, well, ratings, to,..."
398,"After a snowstorm, the roads are blocked and t...","[after, a, snowstorm, ,, the, roads, are, bloc..."
323,Even though I saw this film when I was very yo...,"[even, though, i, saw, this, film, when, i, wa..."
519,I'm the type of guy who loves hood movies from...,"[i, 'm, the, type, of, guy, who, loves, hood, ..."
916,Wow. This is really not that good. <br /><br /...,"[wow, ., this, is, really, not, that, good, .,..."
922,"Once in a while, a movie will sweep along that...","[once, in, a, while, ,, a, movie, will, sweep,..."
5,"Probably my all-time favorite movie, a story o...","[probably, my, all, -, time, favorite, movie, ..."


###  <font color='#6629b2'>Lexicon</font>

Then we need to assemble a lexicon (aka vocabulary) of words that the model needs to know. Each tokenized word in the reviews is added to the lexicon, and then each word is mapped to a numerical index that can be read by the model. Since large datasets may contain a huge number of unique words, it's common to filter all words occurring less than a certain number of times, and replace them with some generic &lt;UNK&gt; token. The min_freq parameter in the function below defines this threshold. When assigning the indices, the number 1 will represent unknown words. The number 0 will represent "empty" word slots, which is explained below. Therefore "real" words will have indices of 2 or higher.

In [26]:
'''Count tokens (words) in texts and add them to the lexicon'''

import pickle

def make_lexicon(token_seqs, min_freq=1, use_padding=False):
    # First, count how often each word appears in the text. Save this count in a dictionary called token_counts
    token_counts = {}
    for word in token_seqs:
      for token in word:
          if token in token_counts:
              token_counts[token] += 1
          else:
              token_counts[token] = 1
 
    # Then, assign each word to a numerical index, i.e save  all these words in a list. Filter words that occur less than or equal to min_freq times.
    lexicon = [token for token, count in token_counts.items() if count >= min_freq]
    
    #create a dictionary lexicon that maps each word to its index. Note that indexes will start from 2,  index 0 is saved for padding and index 1 for unknown words ('<UNK>')
    lexicon = {token:idx + 2 for idx,token in enumerate(lexicon)}
    
    lexicon[u'<UNK>'] = 1 # Unknown words are those that occur fewer than min_freq times
    lexicon_size = len(lexicon)
    print(lexicon)

    print("LEXICON SAMPLE ({} total items):".format(lexicon_size))
    print(dict(list(lexicon.items())[:20]))
    
    return lexicon

lexicon = make_lexicon(token_seqs=train_reviews['Tokenized_Review'], min_freq=1)

{'oh': 2, 'god': 3, ',': 4, 'i': 5, 'must': 6, 'have': 7, 'seen': 8, 'this': 9, 'when': 10, 'was': 11, 'only': 12, '11': 13, 'or': 14, 'twelve': 15, '(': 16, 'do': 17, "n't": 18, 'ask': 19, 'how': 20, ')': 21, 'may': 22, 'been': 23, 'young': 24, 'but': 25, 'stupid': 26, '.': 27, 'anyone': 28, 'could': 29, 'see': 30, 'that': 31, 'is': 32, 'a': 33, 'bad': 34, 'movie': 35, 'nasty': 36, 'gross': 37, 'unscary': 38, 'and': 39, 'very': 40, 'silly': 41, "'ve": 42, 'more': 43, 'impressive': 44, 'effects': 45, 'at': 46, 'disneyland': 47, 'better': 48, 'performances': 49, 'school': 50, 'play': 51, 'convincing': 52, 'crocodiles': 53, 'the': 54, 'zoo': 55, 'where': 56, 'they': 57, 'nothing': 58, 'sit': 59, 'in': 60, 'water': 61, 'ignoring': 62, 'children': 63, 'tapping': 64, 'on': 65, 'glass.<br': 66, '/><br': 67, '/>the': 68, 'story': 69, 'set': 70, 'northern': 71, 'australia': 72, 'handful': 73, 'of': 74, 'ambitious': 75, 'people': 76, 'are': 77, 'trying': 78, 'out': 79, 'new': 80, 'sport': 81, '

###  <font color='#6629b2'>From strings to numbers</font>

Once the lexicon is built, we can use it to transform each review from a list of string tokens into a list of numerical indices.

In [27]:
'''Convert each review from a list of tokens to a list of numbers (indices)'''

def tokens_to_idxs(token_seqs, lexicon): 
    #complete this function to return a list of indexed tokens 
    idx_seqs = [[lexicon[token] if token in lexicon else lexicon['<UNK>'] for token in data] for data in token_seqs]
    return idx_seqs

train_reviews['Review_Idxs'] = tokens_to_idxs(token_seqs = train_reviews['Tokenized_Review'], lexicon = lexicon)
                                   
train_reviews[['Tokenized_Review', 'Review_Idxs']][:10]

,Tokenized_Review,Review_Idxs
82,"[oh, god, ,, i, must, have, seen, this, when, ...","[2, 3, 4, 5, 6, 7, 8, 9, 10, 5, 11, 12, 13, 14..."
991,"[i, dunno, sometimes, ..., you, try, and, try,...","[5, 172, 173, 174, 168, 175, 39, 175, 39, 175,..."
789,"[this, little, two, -, person, movie, is, actu...","[9, 335, 402, 223, 403, 35, 32, 226, 295, 404,..."
894,"[after, several, extremely, well, ratings, to,...","[92, 278, 426, 328, 427, 103, 54, 428, 74, 429..."
398,"[after, a, snowstorm, ,, the, roads, are, bloc...","[92, 33, 484, 4, 54, 485, 77, 486, 39, 54, 487..."
323,"[even, though, i, saw, this, film, when, i, wa...","[308, 401, 5, 196, 9, 114, 10, 5, 11, 40, 24, ..."
519,"[i, 'm, the, type, of, guy, who, loves, hood, ...","[5, 605, 54, 606, 74, 309, 101, 607, 608, 152,..."
916,"[wow, ., this, is, really, not, that, good, .,...","[729, 27, 9, 32, 169, 156, 31, 161, 27, 212, 2..."
922,"[once, in, a, while, ,, a, movie, will, sweep,...","[183, 60, 33, 184, 4, 33, 35, 452, 743, 518, 3..."
5,"[probably, my, all, -, time, favorite, movie, ...","[757, 455, 178, 223, 758, 337, 35, 4, 33, 69, ..."


In [28]:
'''Encode reviews (train_reviews['Review_Idxs']) as bag-of-words vectors'''

import numpy 

def idx_seqs_to_bows(idx_seqs, matrix_length):
    #complete the function to return an array having bag-of-words vectors of the encoded reviews
    # hint: numpy.bincount()
    bow_seqs = numpy.array([numpy.bincount(numpy.array(data), minlength = matrix_length) for data in idx_seqs])
    return bow_seqs

bow_train_reviews = idx_seqs_to_bows(train_reviews['Review_Idxs'], 
                                     matrix_length=len(lexicon) + 1) #add one to length for padding)

print("TRAIN INPUT:\n", bow_train_reviews)
print("SHAPE:", bow_train_reviews.shape, "\n")

#Showing an example mapping string words to counts
lexicon_lookup = {idx: lexicon_item for lexicon_item, idx in lexicon.items()}
lexicon_lookup[0] = ""
pd.DataFrame([(lexicon_lookup[idx], count) for idx, count in enumerate(bow_train_reviews[0])], 
                 columns=['Word', 'Count'])

TRAIN INPUT:
 [[0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 1 1]]
SHAPE: (750, 17068) 



,Word,Count
0,,0
1,<UNK>,0
2,oh,1
3,god,1
4,",",20
...,...,...
17063,upgrade,0
17064,upgraded,0
17065,meter,0
17066,copter,0


##  <font color='#6629b2'>Building a Recurrent Neural Network </font>



###  <font color='#6629b2'>Numerical lists to matrices</font>

The input representation for the RNN explicitly encodes the order of words in the review. We'll return to the lists of the word indices contained in train_reviews['Review_Idxs']. The input to the model will be these number sequences themselves. We need to put all the reviews in the training set into a single matrix, where each row is a review and each column is a word index in that sequence. This enables the model to process multiple sequences in parallel (batches) as opposed to one at a time. Using batches significantly speeds up training. However, each review has a different number of words, so we create a padded matrix equal to the length on the longest review in the training set. For all reviews with fewer words, we prepend the row with zeros representing an empty word position. This is why the number 0 was not assigned as a word index in the lexicon. We can tell Keras to ignore these zeros during training.

In [29]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def pad_idx_seqs(idx_seqs):
    
    #find the biggest review's length and save it in the variable below
    max_seq_len = max([len(idx_seq) for idx_seq in idx_seqs])
    
    #pad all these indexed reviews and return these padded sequences
    #HINT: use pad_sequences function by keras
    padded_idxs = pad_sequences(sequences=idx_seqs, maxlen=max_seq_len)
    return padded_idxs


train_padded_idxs = pad_idx_seqs(train_reviews['Review_Idxs'])

print("TRAIN INPUT:\n", train_padded_idxs)
print("\nSHAPE:", train_padded_idxs.shape, "\n")

TRAIN INPUT:
 [[    0     0     0 ...   126   127    27]
 [    0     0     0 ...     4   401    27]
 [    0     0     0 ...    33   424   425]
 ...
 [    0     0     0 ... 11827 17026   926]
 [    0     0     0 ...     4  3108    27]
 [    0     0     0 ...   161  1454    27]]

SHAPE: (750, 1446) 



###  <font color='#6629b2'>Model Layers</font>
The RNN will have four layers:

**1. Input**: The input layer takes in the matrix of word indices.

**2. Embedding**: A [layer](https://keras.io/layers/embeddings/) that converts integer word indices into distributed vector representations (embeddings), which were introduced above. The difference here is that rather than plugging in embeddings from a pretrained model as before, the word embeddings will be learned inside the model itself. Thus, the input to the model will be the word indices rather than their embeddings, and the embedding values will change as the model is trained. The mask_zero=True parameter in this layer indicates that values of 0 in the matrix (the padding) will be ignored by the model.

**3. GRU**: A [recurrent (GRU) hidden layer](https://keras.io/layers/recurrent/), the central component of the model. As it observes each word in the review, it integrates the word embedding representation with what it's observed so far to compute a representation (hidden state) of the review at that timepoint. There are a few architectures for this layer - we use the GRU variation, Keras also provides LSTM or just the simple vanilla recurrent layer (see the materials at the bottom for an explanation of the difference). This layer outputs the last hidden state of the sequence (i.e. the hidden representation of the review after its last word is observed).

**4. Dense**: An output [layer](https://keras.io/layers/core/#dense) that predicts the rating for the review based on its GRU representation given by the previous layer. It has one dimension that contains a continuous value (the rating). Add a proper activation function.

###  <font color='#6629b2'>Parameters</font>

Our function for creating the RNN takes the following parameters:

**n_input_nodes**: As with the standard bag-of-words MLP, this is the number of unique words in the lexicon, plus one to account for the padding represented by 0 values. This indicates the number of rows in the embedding layer, where each row corresponds to a word.

**n_embedding_nodes**: the number of dimensions (units) in the embedding layer, which can be freely defined. Here, it is set to 300.

**n_hidden_nodes**: the number of dimensions in the GRU hidden layer. Like the embedding layer, this can be freely chosen. Here, it is set to 500.

In [30]:
'''Create the model'''

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.layers import InputLayer

def create_rnn_model(n_input_nodes, n_embedding_nodes, n_hidden_nodes):
    
    #complete this function to create a model and compile it having the 4 layers listed above.
    #Note: Layer 1 -  Technically the shape of this layer is (batch_size, len(train_padded_idxs)).
    # However, both the batch size and the length of the input matrix can be inferred from the input at training time. 
    # The batch size is implicitly included in the shape of the input, so it does not need to 
    # be specified as a dimension of the input. None can be given as placeholder for the input matrix length.
    # By defining it as None, the model is flexible in accepting inputs with different lengths.
    #model = model.Sequential()
    input_layer = Input(shape=(None,))
    
    # Layer 2
    embedding_layer = Embedding(input_dim = n_input_nodes, output_dim = n_embedding_nodes, mask_zero = True)(input_layer)
    
    # Layer 3
    gru_layer = GRU(units=n_hidden_nodes)(embedding_layer)

    # Layer 4
    output_layer = Dense(units=1, activation='sigmoid')(gru_layer)
    
    #Specify which layers are input and output, compile model with loss and optimization functions
    model = Model(inputs=[input_layer], outputs=output_layer)
    model.compile(loss="mean_squared_error", optimizer='adam')
    
    return model


In [31]:
rnn_model = create_rnn_model(n_input_nodes=len(lexicon) + 1, n_embedding_nodes=300, n_hidden_nodes=500)

###  <font color='#6629b2'>Training</font>

In [32]:
'''
Train the rnn_model using the padded sequences and y=train_reviews['Rating'].['Review_Idxs']
You need  to convert train_reviews['Rating'] to tensor before passing it as an argument
Hint: tf.convert_to_tensor
batch_size=20, epochs=5
'''
import tensorflow as tf
rnn_model.fit(x=(train_padded_idxs), y=tf.convert_to_tensor(train_sentiment['sentiment']), batch_size=20, epochs=5)

Epoch 1/5
38/38 [==============================] - 446s 12s/step - loss: 0.2486
Epoch 2/5
38/38 [==============================] - 450s 12s/step - loss: 0.1742
Epoch 3/5
38/38 [==============================] - 444s 12s/step - loss: 0.0574
Epoch 4/5
38/38 [==============================] - 441s 12s/step - loss: 0.0359
Epoch 5/5
38/38 [==============================] - 440s 12s/step - loss: 0.0186


###  <font color='#6629b2'>Prediction</font>

In [37]:
'''Put test reviews in padded matrix just how we did for train_reviews'''#text_to_tokens(train_reviews['review'])
# test_reviews['Review_Idxs'] = tokens_to_idxs(token_seqs = test_reviews['Tokenized_Review'], lexicon=rnn_lexicon)
test_reviews['Review_Idxs'] = tokens_to_idxs(token_seqs = text_to_tokens(test_reviews['review']), lexicon=lexicon)
test_padded_idxs = pad_idx_seqs(test_reviews['Review_Idxs'])

print("TEST INPUT:\n", test_padded_idxs)
print("SHAPE:", test_padded_idxs.shape, "\n")

TEST INPUT:
 [[    0     0     0 ... 15387   194    27]
 [    0     0     0 ...   274   454   926]
 [    0     0     0 ...  1415   875    27]
 ...
 [    0     0     0 ...  6990 13040    27]
 [    0     0     0 ...    67  1200  1692]
 [    0     0     0 ...    31  2133    27]]
SHAPE: (250, 1241) 



In [38]:
'''Predict the ratings '''

#Since ratings are integers, need to round predicted rating to nearest integer
test_reviews['RNN_Pred_Rating'] = numpy.round(rnn_model.predict(test_padded_idxs)[:,0]).astype(int)

###  <font color='#6629b2'>Evaluation</font>

In [46]:
'''Evaluate the model with R^2'''#tf.convert_to_tensor(test_reviews)

# print the r2 score
from sklearn.metrics import r2_score 
r2 = r2_score(y_true=test_sentiment, y_pred=test_reviews['RNN_Pred_Rating'])
print("COEFFICIENT OF DETERMINATION (R2): {:.3f}".format(r2))

COEFFICIENT OF DETERMINATION (R2): -0.970


On the full test dataset of 25,000 reviews, the $R^2$ for this model is 0.622525. So the RNN outperforms the continuous bag-of-words MLP as well as the standard bag-of-words approach.
Your score might not be good because we're training on only 100-200 reviews.

### <font color='#6629b2'>Visualizing data inside the model</font>

To help visualize the data representation inside the model, we can look at the output of each layer in a model individually. Keras' Functional API lets you derive a new model with the layers from an existing model, so you can define the output to be a layer below the output layer in the original model. Calling predict() on this new model will produce the output of that layer for a given input. Of course, glancing at the numbers by themselves doesn't provide any interpretation of what the model has learned (although there are opportunities to [interpret these values](https://medium.com/civis-analytics/interpreting-and-visualizing-neural-networks-for-text-processing-e9dff0da9c22), but seeing them verifies the model is just a series of transformations from one matrix to another. The model stores its layers as the list model.layers, and you can retrieve specific layer by its position index in the model.

In [40]:
'''Showing the output of the RNN embedding layer (second layer) for the test reviews'''

embedding_layer = Model(inputs=rnn_model.layers[0].input, 
                        outputs=rnn_model.layers[1].output) #embedding layer is 2nd layer (index 1)
embedding_output = embedding_layer.predict(test_padded_idxs)
print("EMBEDDING LAYER OUTPUT SHAPE:", embedding_output.shape)
print(embedding_output[0])

EMBEDDING LAYER OUTPUT SHAPE: (250, 1241, 300)
[[-0.03758577 -0.01309761  0.04766381 ...  0.005745    0.00612906
   0.04314497]
 [-0.03758577 -0.01309761  0.04766381 ...  0.005745    0.00612906
   0.04314497]
 [-0.03758577 -0.01309761  0.04766381 ...  0.005745    0.00612906
   0.04314497]
 ...
 [-0.0275459  -0.03493754  0.02585259 ... -0.02959081  0.01951958
  -0.02096698]
 [-0.03299135  0.00254508 -0.03437503 ... -0.03742383  0.01111755
   0.02982102]
 [-0.01937766 -0.00305706 -0.01363741 ... -0.01971357 -0.0031043
  -0.00032688]]


## <font color='#6629b2'>Conclusion</font>

As mentioned above, the models shown here could be applied to any task where the goal is to predict a score for a particular sequence. For ratings prediction, this output is ordinal, but it could also be categorical with a few simple changes to the output layer of the model. 

## <font color='#6629b2'>More resources</font>

Yoav Goldberg's book [Neural Network Methods for Natural Language Processing](http://www.morganclaypool.com/doi/abs/10.2200/S00762ED1V01Y201703HLT037) is a thorough introduction to neural networks for NLP tasks in general.

If you'd like to learn more about what Keras is doing under the hood, there is a [Theano tutorial](http://deeplearning.net/tutorial/lstm.html) that also applies an RNN to sentiment prediction, using the same dataset here

Andrej Karpathy's blog post [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) is very helpful for understanding the mathematical details of an RNN, applied to the task of language modeling. It also provides raw Python code with an implementation of the backpropagation algorithm.

TensorFlow also has an RNN language model [tutorial](https://www.tensorflow.org/versions/r0.12/tutorials/recurrent/index.html) using the Penn Treebank dataset

Chris Olah provides a good [explanation](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) of how LSTM RNNs work (this explanation also applies to the GRU model used here)

Denny Britz's [tutorial](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) documents well both the technical details of RNNs and their implementation in Python.